In [ ]:
# https://dotblogs.com.tw/chris0920/2010/10/25/18560
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy.misc import imread
import glob
import random
import multiprocessing
from sklearn.cluster import DBSCAN
import cv2
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
num_select = 500
img_file = sorted(glob.glob('../input/train/*/*.jpg'),
                  key=lambda x:random.random())[:num_select]
img = np.array([imread(img) for img in img_file])
print('Image Length : %d ' %(len(img)))

## Image Size

In [ ]:
img_shape = np.array([str(img_.shape) for img_ in img])
print('Image Shape Count : \n')
print(pd.Series(img_shape).value_counts())
# width height channel

In [ ]:
def show_img(img_,title):
    selected_img_idx = np.random.choice(len(img_),4) ## random select image 
    fig,axes = plt.subplots(1,4,sharex='col',sharey='row',figsize=(12,3))
    plt.suptitle(title,size=20)
    for i,idx in enumerate(selected_img_idx):
        axes[i].imshow(img_[idx])
   
    

In [ ]:
for idx in pd.Series(img_shape).unique():
    show_img(img[img_shape == idx],'Image Shape : {} '.format(idx))
    plt.show()
    

## boart clustering with distance matrix

#### cv2 resize : http://monkeycoding.com/?p=609

In [ ]:
def normalize_distance(args):
    img1,img2 = args
    img1= (img1 - img1.mean()) / img1.std() ## normalize = (value - mean) /standard deviation
    img2 = (img2 - img2.mean()) / img2.std() ## normalize = (value - mean)/standard deviation
    return (np.mean(np.abs(img1-img2)))


img_list = [cv2.resize(img_,(224,224),cv2.INTER_LINEAR) for img_ in img]
distance = np.zeros((len(img_list),len(img_list))) ## 500 * 500 dimension because the length of image is 500

# Create the distance matrix in a multithreaded fashion
pool = multiprocessing.Pool(8)
#bar = progressbar.ProgressBar(max=len(train))
distances = np.zeros((len(img_list), len(img_list)))
for i, img_ in enumerate(img_list): #enumerate(bar(train)):
    all_imgs = [(img_, f) for f in img_list]
    dists = pool.map(normalize_distance, all_imgs)
    distances[i, :] = dists

In [ ]:
print(distances)
plt.hist(distances.flatten(),bins=50)

### DBSCAN : http://www.dataivy.cn/blog/dbscan/

In [ ]:
dbscan = DBSCAN(eps=0.5,metric='precomputed',min_samples=5)
y_pred = dbscan.fit_predict(distances)
print(y_pred)
print('Cluster Size Count:\n')
print(pd.Series(y_pred).value_counts())

## Display clustering boat

In [ ]:
for pred in pd.Series(y_pred).unique():
    if pred != -1:
        #len(np.array(img_list)[y_pred == pred])
        show_img(np.array(img_list)[y_pred == pred],'Boat ID {}'.format(pred))
        plt.show()

In [ ]:
i